In [1]:
import base64
import datetime
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import binascii
import akshare as ak
import numpy as np
from datetime import datetime, timedelta
#三低因子策略plus（5只日轮动）双低因子*5,剩余规模*5,纯债溢价率*1，负,负，负
#采用禄得网的3低因子方案调整而来。

In [2]:
#集思录数据
USER_NAME = 'ri' # 账户名
PASSWORD = '9' # 密码
# 定义常量
headers = {
    'authority': 'www.jisilu.cn',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'zh-CN,zh-TW;q=0.9,zh;q=0.8,en;q=0.7',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'dnt': '1',
    'origin': 'https://www.jisilu.cn',
    'pragma': 'no-cache',
    'referer': 'https://www.jisilu.cn/account/login/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82',
    'x-requested-with': 'XMLHttpRequest',
}
AES_KEY = '397151C04723421F'

In [3]:
# 加密函数
def get_encryt_sting(text):
    data = text
    key = AES_KEY
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    encrypted_text = cipher.encrypt(pad(data.encode(), AES.block_size))
    return binascii.hexlify(encrypted_text).decode()

In [4]:
# 获取cookie函数
def get_jsl_cookies(username, password):
    data = {
        'return_url': 'https://www.jisilu.cn/',
        'user_name': get_encryt_sting(username),
        'password': get_encryt_sting(password),
        'auto_login': '0',
        'aes': '1',
    }
    response = requests.post('https://www.jisilu.cn/webapi/account/login_process/', headers=headers, data=data)
    
    return response.cookies
cookies = get_jsl_cookies(USER_NAME, PASSWORD) # 根据用户名密码获取集思录cookie

In [5]:
#开始抓取JSL数据，修改列名，删除不用字段
url='https://www.jisilu.cn/data/cbnew/cb_list_new/'
response = requests.get(url, cookies=cookies, headers=headers)
data = response.json()
df = pd.DataFrame(data["rows"])["cell"].apply(pd.Series)
df = df.rename(
        columns={
            "bond_id": "代码",
            "bond_nm": "转债名称",
            "price": "现价",
            "increase_rt": "涨跌幅",
            "stock_id": "正股代码",
            "stock_nm": "正股名称",
            "sprice": "正股价",
            "sincrease_rt": "正股涨跌",
            "pb": "正股PB",
            "convert_price": "转股价",
            "convert_value": "转股价值",
            "premium_rt": "转股溢价率",
            "dblow": "双低",
            "rating_cd": "评级",
            "put_convert_price": "回售触发价",
            "force_redeem_price": "强赎触发价",
            "convert_amt_ratio": "转债流通市值占比",
            "maturity_dt": "到期时间",
            "year_left": "剩余年限",
            "curr_iss_amt": "剩余规模",
            "volume": "成交额万",
            "turnover_rt": "换手率",
            "ytm_rt": "到期税前收益",
            "ref_yield_info": "贴现说明",
            "convert_dt": "转股开始日",
            "bond_premium_rt": "债底溢价率",
            "adjust_condition": "赎回状态",
            "bond_value": "纯债价值",
            "list_dt": "上市日期",
            "option_value": "期权价值",
            "volatility_rate": "正股年化波动率",
            "fund_rt": "机构持仓",
            "year_left_text": "到期年限",
            "svolume": "正股成交额",
            "bond_nm_tip": "重要提示",
            "orig_iss_amt": "发行规模-亿",
            
        },  # 对索引进行重命名
    )

 # 删除特定的列
df = df.drop(columns=['bond_py','stock_py','btype','sw_cd','qflag2','market_cd','owned','put_ytm_rt','notes','noted','redeem_icon','last_time','qstatus',
                      'hold','price_tips','margin_flg','sqflag','pb_flag','adj_cnt','adj_scnt','convert_price_valid','convert_price_tips','convert_cd_tip',
                     'adjusted','redeem_dt','option_tip','adjust_status','unadj_cnt','adjust_remain_days','adjust_orders','real_force_redeem_price'])       


#——————————————————去集思录的定制页面的数据取另外一些数据
url='https://www.jisilu.cn/webapi/cb/list/'
response = requests.get('https://www.jisilu.cn/webapi/cb/list/', cookies=cookies, headers=headers) # 获取集思录表格数据
df2=pd.DataFrame(response.json().get('data')) # 打印表格数据
# 使用map函数更新df1的'列2'
df['转债波动率'] = df['代码'].map(df2.set_index('bond_id')['bond_stdevry'])

#-------------
 #部份数据因为%的原因要除以100修正。
df['代码']= df['代码'].astype('int64')
df['涨跌幅']=(df['涨跌幅']/100).round(4)
df['正股涨跌']=(df['正股涨跌']/100).round(4)
df['转股溢价率']=(df['转股溢价率']/100).round(4)
df['债底溢价率']=(df['债底溢价率']/100).round(4)
df['正股年化波动率']=(df['正股年化波动率']/100).round(4)  
df['机构持仓']=(df['机构持仓']/100).round(4) 
df['换手率']=(df['换手率']/100).round(4) 
df['上市日期'] = pd.to_datetime(df['上市日期'], format='%Y-%m-%d')
df['转债波动率']=(df['转债波动率']/100).round(4) 
df['剩余年限'] = df['剩余年限'].replace(np.nan, 0) #替换nan为0
df['到期年限'] = df['到期年限'].replace("-", 0)#替换nan为0
df['到期税前收益'] = df['到期税前收益'].replace(np.nan, 0)#替换nan为0

In [6]:
#计算期权价值
from math import log, sqrt, exp
from scipy import stats
# 期权的定价计算,根据公式１
np.seterr(divide='ignore', invalid='ignore')  # 消除被除数为0的警告
def bsm_call_value(S_0, K, T, r, sigma):
    S_0 = float(S_0)
    d_1 = (log(S_0 / K) + (r + 0.5 *sigma **2) *T)/(sigma * sqrt(T))
    d_2 = (log(S_0 / K) + (r - 0.5 *sigma **2) *T)/(sigma * sqrt(T))
    C_0 = (S_0 * stats.norm.cdf(d_1, 0.0, 1.0) - K * exp(-r * T) * stats.norm.cdf(d_2, 0.0, 1.0))
    return C_0

#获取中债国债收益率曲线1年做为无风险收益, 取最近的20条平均值
today = datetime.now() #当前日期
ten_days_ago = today - timedelta(days=10)
bond_china_yield_df = ak.bond_china_yield(start_date=ten_days_ago.strftime('%Y%m%d'), end_date=today.strftime('%Y%m%d'))
bond_china_yield_df =bond_china_yield_df[(bond_china_yield_df['曲线名称']=='中债国债收益率曲线')]
r=round(bond_china_yield_df['1年'].mean()/100,4) #最近10天的平均值，平滑
print("r=",r)
# 计算的一些初始值
#S_0 = df.正股价    # 股票或指数初始的价格;
#K = df.转股价       #  行权价格
#T = df.剩余年限        #  期权的到期年限(距离到期日时间间隔)
#r =        #   无风险利率


cv=[] #生成一个空的list
for i in range(df.shape[0]):
    tmp = df.iloc[i]
    res = bsm_call_value(tmp.正股价,tmp.转股价,tmp.剩余年限, r,tmp.正股年化波动率)
    cv.append(round(res, 4))
    #print(cv)

cv = pd.DataFrame(cv)
index = df.index
cv.set_index(index, inplace=True)
cv.columns=['cv']
df['期权价值']=round(100/df['转股价']*cv['cv'],4)
#根据期权价值计算转债的理论价值和偏离度
df['偏离度'] = -((df['纯债价值']+df['期权价值']) / df['现价'] - 1).round(4)

print("计算期权价值和偏离度完成")

/Users/shao/anaconda3/envs/newbase/lib/python3.11/site-packages/akshare/bond/china_bond.py:160: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_df = pd.read_html(data_text, header=0)[1]


r= 0.0238
计算期权价值和偏离度完成


In [10]:
#import openpyxl
from openpyxl import load_workbook
# 需按照实际策略修改打分因子（三因子排序法-价格-溢价率-波动差）
#通过过滤条件和处理数据 ST ST* R EB Z和交换债
#_--------------
df3d=df #复制一个新表用来计算3低因子的评分
df3d['filter'] = False # 初始化过滤器
df3d.loc[df3d.重要提示.isin(['已满足强赎条件','公告要强赎','临近到期','公告实施强赎', '公告提示强赎', '已满足强赎条件','最后交易日']), 'filter'] = True # 排除赎回状态
df3d.loc[df3d.正股名称.str.contains('ST','*ST'),'filter'] = True  # 排除正股ST的转债
df3d.loc[df3d.正股名称.str.contains('R','*'),'filter'] = True  # R和*的正股
df3d.loc[df3d.转债名称.str.contains('EB'),'filter'] = True  # 可交换债和Z债
df3d.loc[df3d.转债名称.str.contains('Z','退'),'filter'] = True  # 转债退市
df3d.loc[df3d.成交额万 ==0, 'filter'] = True # 排除未上市的新债
df3d = df3d[~df3d['filter']] #删除
df3d=df3d.reset_index(drop=True)  
#------------------
#以下的过滤为通用设置，如果是与排除因子和打分因子放到具体的模块
df3d['上市天数'] = df3d['上市日期'].rsub(pd.Timestamp('today')).dt.days
df3d.loc[df3d.上市天数 <=3, 'filter'] = True # 排除上市3天内新债
df3d.loc[df3d.评级.isin(['BBB+','BBB','BBB-','B+','B','B-','cc','ccc']),'filter'] = True #排除低级债 
#df3d.loc[df3d.rating.isin(['8-','9','10']),'filter'] = True #排除低级债 

#--------------------
#排除因子
df3d.loc[df3d.转股价值 > 130, 'filter'] = True # 排除转股价值大于130的标的
df3d.loc[df3d.现价 < 100, 'filter'] = True # 排除转股价值小于100的标的
df3d.loc[df3d.剩余年限 < 1 , 'filter'] = True # 排除剩余年限小于1年的标的
df3d['成交额万pct'] = df3d['成交额万'].rank(pct=True) # 将成交额从小到大百分比排列
df3d.loc[df3d.成交额万pct < 0.05, 'filter'] = True # 排除换成交额排名小于3%的标的
#--------------------
#打分因子  	
df3d['双低分'] =df3d.loc[df3d['filter'] == False, '双低'].rank(ascending=False) # 对打分因子1进行打分
df3d['规模分'] = df3d.loc[df3d['filter'] == False, '剩余规模'].rank(ascending=False) # 对打分因子2进行打分
df3d['债溢分'] = df3d.loc[df3d['filter'] == False, '债底溢价率'].rank(ascending=False) # 对打分因子3 进行打分
df3d['总分'] =df3d.双低分 * 5 + df3d.规模分 * 5+df3d.债溢分           #计算总分，权重为：双低因子*5,剩余规模*5,纯债溢价率*1，负,负，负，需按照实际策略进行修改
df3d['排名'] = df3d['总分'].rank(ascending=False)         # 按总分从高到低计算排名
df3d.sort_values('排名', ascending=True, inplace=True)  # 根据 '排名' 列进行升序排序
#df.insert(0, '排名', df.pop('排名'))                   
df3d=df3d[['代码','转债名称','现价','转股溢价率', '涨跌幅','正股涨跌','成交额万','剩余规模','转股价值', '纯债价值','期权价值','债底溢价率','偏离度','转债波动率',
           '正股年化波动率','换手率','剩余年限','双低','评级','赎回状态','总分','filter','重要提示']]

#写文件

with pd.ExcelWriter(r'可转债量化轮动.xlsx',engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
    df3d.to_excel(writer,  sheet_name='三低组',index=False, header=True)
    df3d.head(20).to_excel(writer, sheet_name='持仓明细',columns=['代码','转债名称','现价','转股溢价率', '涨跌幅','正股涨跌','成交额万','剩余规模','纯债价值',
                                                              '期权价值','偏离度','换手率'],startrow=35,startcol=10,index=False, header=True)  #同时运行这一行会文件会报错。
    
print("最新数据更新成功！！！！"+str(pd.Timestamp('today')))




最新数据更新成功！！！！2023-12-12 10:51:10.840428
